In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint

In [2]:
import json

In [3]:
# Import API key
# from api_keys import audio_api

In [3]:
#create an empty list to populate with artist names
artist_names=[]

In [4]:
#generate random numbers in the range of available artist IDs
#used to populate artist_names list
#https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html
random_nums = np.random.randint(low=100000, high=170000, size=800)

#print(random_nums)

#test with known IDs and intentional errors
#random_nums = [112024, 0, 100000, 114364]

In [5]:
#create a base url
#example: https://theaudiodb.com/api/v1/json/1/artist.php?i=112024
id_url = "https://theaudiodb.com/api/v1/json/1/artist.php?i="

#create a loop that uses random number
for num in random_nums:
    unique_url = id_url + f'{num}'
    
    #call api url
    request = requests.get(unique_url)
    
    #for each attempt try to convert information to json
    try:
        #convert to json
        info = request.json()
    
    #exception json decode error
    #https://docs.python.org/3/tutorial/controlflow.html
    except json.JSONDecodeError:
        #end this iteration and continue new iteration of for loop
        continue
    
    #if json conversion successful then
    #try to create a variable for single artist to reference later
    try:
        artist = info['artists'][0]['strArtist']
        
    #TypeError exception if url responds with {'artists': None}
    #https://docs.python.org/3/tutorial/errors.html
    except (TypeError):
        continue
        
    #if the artist is not already in the list then 
    if artist not in artist_names:
        
        #populate artist_names list using info(json)
        #response-> {'artists': [{'idArtist': '114364', 'strArtist': 'Beyoncé', ...
        #dictionary{'artists':['{dictionary}']}
        artist_names.append(info['artists'][0]['strArtist'])
    
#print(artist_names)

In [6]:
len(artist_names)

612

In [8]:
#print(artist_names)

In [9]:
#alternative to "if not in list then .append"
#convert list to set to remove redundant artists
#https://java2blog.com/python-list-to-set/


#unique_artist_ids = set(artist_names)
#print(unique_artist_ids)

In [7]:
#create loop to pull artist information from the database
#using populated artist_names list
#example url: http://theaudiodb.com/api/v1/json/1/search.php?s=coldplay
#tutorial: https://www.youtube.com/watch?v=pxofwuWTs7c

#create empty dictionary to fill with values
audio_data = {'artist_id':[],
              'artist_name':[],
              'gender':[],
              'members':[],
              'style':[], 
              'genre':[],
             'year_formed':[],
             'year_disbanded':[],
             'location':[]}

#for each artist in the artist list, 
for name in artist_names:
    
    #add the artist name to the base url
    base_url = "http://theaudiodb.com/api/v1/json/1/search.php?s="
    req = requests.get(base_url + f'{name}')
    
    #convert request information into json as a variable
    data = req.json()
    
    #add json values to dictionary
    try:
        audio_data['artist_id'].append(data['artists'][0]['idArtist'])
        audio_data['artist_name'].append(data['artists'][0]['strArtist'])
        audio_data['gender'].append(data['artists'][0]["strGender"])
        audio_data['members'].append(data['artists'][0]["intMembers"])
        audio_data['style'].append(data['artists'][0]["strStyle"])
        audio_data['genre'].append(data['artists'][0]["strGenre"])
        audio_data['year_formed'].append(data['artists'][0]["intFormedYear"])
        audio_data['year_disbanded'].append(data['artists'][0]["strDisbanded"])
        audio_data['location'].append(data['artists'][0]["strCountry"])
    
    #restart loop if TypeError found
    except (TypeError):
        continue

In [11]:
#print(audio_data)

In [8]:
audio_df = pd.DataFrame(audio_data)
audio_df.head(20)

,artist_id,artist_name,gender,members,style,genre,year_formed,year_disbanded,location
0,123895,Fiction Factory,Male,4,Rock/Pop,New Wave,1983,None,"Perth, Scotland, UK"
1,167035,Dorothy Masuka,Female,1,,,None,None,
2,116208,Acoustic Blues Brothers,Male,2,Blues,Acoustic,1996,Yes,"Zurich, Switzerland"
3,134697,Reaching Quiet,None,None,None,,0,None,None
4,139142,Axamenta,None,None,None,Metal,1993,None,None
5,158789,Four80East,None,None,None,Acid Jazz,1997,None,None
6,115846,Chairlift,Mixed,2,Rock/Pop,Electronic,2005,None,"Boulder, Colorado (USA)"
7,148576,Brave Old World,,1,,Folk,1989,None,
8,125523,Church of Misery,Male,4,,Doom Metal,1995,None,"Tokyo,Japan"
9,166417,Radkey,Male,1,,,2010,None,"St. Joseph, Missouri"


In [42]:
# Cleaning the data: replace the 0s, None, and empty strings with null. Year disbanded: replaced None to no, meaning that they haven't disbanded
#To only replace empty values for one column, specify the column name for the DataFrame:
#df["Calories"].fillna(130, inplace = True)

audio_df['year_disbanded'].replace({2021 : 'No'}, inplace = True)
#audio_df_nan = audio_df.replace(0, np.nan)
audio_df['year_formed'].replace({None: 'Null', '0' : np.nan}, inplace = True) 
audio_df['gender'].replace({None: 'Null', '' : 'Null'}, inplace = True) 
audio_df['location'].replace({None: 'Null', '' : 'Null'}, inplace = True)
audio_df['style'].replace({'' : 'Null', None : 'Null'}, inplace = True)

#cols = ['year_formed',"gender","BootSize","SuitSize","Type"]
#df2[cols] = df2[cols].replace({'0':np.nan, 0:np.nan})
audio_df.head(20)

,artist_id,artist_name,gender,members,style,genre,year_formed,year_disbanded,location
0,123895,Fiction Factory,Male,4,Rock/Pop,New Wave,1983,No,"Perth, Scotland, UK"
1,167035,Dorothy Masuka,Female,1,Null,,Null,No,Null
2,116208,Acoustic Blues Brothers,Male,2,Blues,Acoustic,1996,Yes,"Zurich, Switzerland"
3,134697,Reaching Quiet,Null,None,Null,,Null,No,Null
4,139142,Axamenta,Null,None,Null,Metal,1993,No,Null
5,158789,Four80East,Null,None,Null,Acid Jazz,1997,No,Null
6,115846,Chairlift,Mixed,2,Rock/Pop,Electronic,2005,No,"Boulder, Colorado (USA)"
7,148576,Brave Old World,Null,1,Null,Folk,1989,No,Null
8,125523,Church of Misery,Male,4,Null,Doom Metal,1995,No,"Tokyo,Japan"
9,166417,Radkey,Male,1,Null,,2010,No,"St. Joseph, Missouri"


In [13]:
#possible questions
#location and genre
#year formed and genre

#pull artist, location, year formed, genre

In [14]:
#audio_df_nan = audio_df.replace('', np.nan)
#artist_df = audio_df_nan.dropna(axis=0, how='any')
#artist_df

In [ ]:
# Percentage of artists how are Urban/R&B
#Count of artists per style
